In [1]:
## Import Libraries, and read the key-token
import openai
import os
from dotenv import load_dotenv
import os

## Load the Key token
_ = load_dotenv()
key_token = os.getenv('OpenAI_KEY_TOKEN')

## Assign that key_token to api_key of OpenAI
openai.api_key = key_token

In [2]:
# ## Embeddings
# def get_embedding(text, model="text-embedding-ada-002"):
#     text = text.replace("\n", " ")
#     response = openai.Embedding.create(input=[text], model=model)
#     response = response['data'][0]['embedding']
#     return response

In [5]:
# get_embedding(text='Mohammed Agoor', model="text-embedding-ada-002")

1536

In [41]:
##
from openai.embeddings_utils import get_embedding, cosine_similarity
import pandas as pd
import tiktoken
import numpy as np

In [ ]:
get_embedding(text='Mohammed Agoor', engine='text-embedding-ada-002')

In [21]:
df = pd.read_csv('example.csv', header=None, names=['text'])
df

,text
0,red
1,blue
2,dog
3,cat
4,fish
5,chicken
6,meat
7,egg
8,bread
9,ice


In [22]:
## Count the cost of embedding this df
enc = tiktoken.encoding_for_model('text-embedding-ada-002')
enc.encode(df['text'].iloc[0]) ## the fist instance in the df

[1171]

In [25]:
total_tokens = sum([len(enc.encode(word)) for word in df['text']])
total_tokens

23

In [26]:
## Price in USD
cost = total_tokens * (0.0004 / 1000)
cost

9.2e-06

In [28]:
## Embedding for these words
# get_embedding(df['text'].iloc[0])

## apply for all words
df['embeddings'] = df['text'].apply(lambda word: get_embedding(word, engine='text-embedding-ada-002'))

In [32]:
## store embeddings for cachign and saving money for real time inference
df.to_csv('embeddings.csv', index=False)

In [44]:
## laod it 
df_embed = pd.read_csv('embeddings.csv')

## Convert from string to numpy array
df_embed['embeddings'] = df_embed['embeddings'].apply(eval).apply(np.array)

In [52]:
## Searching in real time
search_word = 'red'  ## --> I want to get similar words to this one --> First get its embedding vector
search_vector = get_embedding(text=search_word, engine='text-embedding-ada-002')

## Get the simialr using metric (cosing_similarity) for example.
## Apply that for each instance
df_embed['similarity'] = df_embed['embeddings'].apply(lambda data_vector: cosine_similarity(data_vector, search_vector))

## Sorting
df_embed = df_embed.sort_values(by='similarity', ascending=False)


## Use Threshold to filter these.
df_embed[df_embed['similarity'] > 0.3]['text'].tolist()

['red',
 'blue',
 'bad',
 'good',
 'water',
 'bread',
 'google',
 'egg',
 'chicken',
 'food',
 'dog',
 'drink',
 'ice',
 'fish',
 'meat',
 'microsoft',
 'facebook',
 'cat',
 'youtube',
 'amazon']